# 🍽️ Projet V9 - OPTIMISÉ RECHERCHE WEB

**Optimisations basées sur la recherche :**
- ✅ Mean Pooling (au lieu de [CLS])
- ✅ Mixed Precision (fp16)
- ✅ Learning rate 3e-5
- ✅ Hidden dim 384
- ✅ 4 epochs + patience 2

| Version | Temps | Accuracy |
|---------|-------|----------|
| V8 | 29 min | 87.94% |
| **V9** | **~30-35 min** | **~89-90%** |

In [ ]:
!pip install -q transformers torch pandas numpy tqdm accelerate

In [ ]:
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"FP16 support: {torch.cuda.is_bf16_supported()}")

In [ ]:
# ============================================
# CONFIG V9 - OPTIMISÉ
# ============================================
CONFIG = {
    "model_name": "camembert/camembert-large",
    "num_epochs": 4,          # +1 epoch
    "batch_size": 12,
    "learning_rate": 3e-5,    # 3e-5 au lieu de 2e-5
    "max_length": 256,
    "dropout": 0.1,
    "label_smoothing": 0.1,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "hidden_dim": 384,        # 384 au lieu de 256
    "patience": 2,            # 2 au lieu de 1
    "use_fp16": True,         # Mixed precision
    "use_mean_pooling": True, # Mean pooling
}
print("📋 Config V9:")
for k, v in CONFIG.items(): print(f"  {k}: {v}")

In [ ]:
import os
from google.colab import files
os.makedirs('/content/data', exist_ok=True)
print("Uploadez ftdataset_train.tsv et ftdataset_val.tsv:")
for f in files.upload().keys(): os.rename(f, f'/content/data/{f}')
!ls /content/data/

In [ ]:
import torch, torch.nn as nn, pandas as pd, time
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from transformers import AutoTokenizer, AutoModel, get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

LABEL_TO_IDX = {"Positive": 0, "Négative": 1, "Neutre": 2, "NE": 3}
IDX_TO_LABEL = {v: k for k, v in LABEL_TO_IDX.items()}
ASPECTS = ["Prix", "Cuisine", "Service"]

class DS(Dataset):
    def __init__(s, t, tok, l=None, ml=256):
        s.e = tok(t, truncation=True, padding=True, max_length=ml, return_tensors="pt")
        s.l = l
    def __len__(s): return len(s.e["input_ids"])
    def __getitem__(s, i):
        it = {"input_ids": s.e["input_ids"][i], "attention_mask": s.e["attention_mask"][i]}
        if s.l:
            for a in ASPECTS: it[f"label_{a.lower()}"] = torch.tensor(s.l[a][i], dtype=torch.long)
        return it

def prep_l(d): return {a: [LABEL_TO_IDX.get(x[a], 3) for x in d] for a in ASPECTS}
def get_t(d): return [x["Avis"] for x in d]
def coll(f):
    b = {"input_ids": torch.stack([x["input_ids"] for x in f]), "attention_mask": torch.stack([x["attention_mask"] for x in f])}
    for a in ASPECTS:
        k = f"label_{a.lower()}"
        if k in f[0]: b[k] = torch.stack([x[k] for x in f])
    return b

class ClassifierV9(nn.Module):
    """Classificateur optimisé avec Mean Pooling"""
    def __init__(s, mn, hd=384, dr=0.1, use_mean_pooling=True):
        super().__init__()
        s.enc = AutoModel.from_pretrained(mn)
        hs = s.enc.config.hidden_size
        s.use_mean_pooling = use_mean_pooling
        
        # Têtes de classification avec couche cachée plus large
        s.cls = nn.ModuleDict({
            a: nn.Sequential(
                nn.Dropout(dr),
                nn.Linear(hs, hd),
                nn.GELU(),
                nn.LayerNorm(hd),  # Ajout LayerNorm
                nn.Dropout(dr),
                nn.Linear(hd, 4)
            ) for a in ASPECTS
        })
    
    def mean_pooling(s, hidden_states, attention_mask):
        """Mean pooling sur les tokens (meilleur que [CLS] seul)"""
        mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        sum_embeddings = torch.sum(hidden_states * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
        return sum_embeddings / sum_mask
    
    def forward(s, ids, mask):
        o = s.enc(input_ids=ids, attention_mask=mask)
        if s.use_mean_pooling:
            pooled = s.mean_pooling(o.last_hidden_state, mask)
        else:
            pooled = o.last_hidden_state[:, 0, :]  # [CLS]
        return {a: s.cls[a](pooled) for a in ASPECTS}

class Trainer:
    def __init__(s, c):
        s.c = c; s.dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"🔧 Chargement de {c['model_name']}...")
        s.tok = AutoTokenizer.from_pretrained(c['model_name'])
        s.m = ClassifierV9(
            c['model_name'], c['hidden_dim'], c['dropout'], c['use_mean_pooling']
        ).to(s.dev)
        s.crit = nn.CrossEntropyLoss(label_smoothing=c['label_smoothing'])
        s.scaler = GradScaler() if c['use_fp16'] else None
        params = sum(p.numel() for p in s.m.parameters())/1e6
        print(f"✅ Modèle sur {s.dev} ({params:.0f}M params)")
        print(f"   Mean Pooling: {c['use_mean_pooling']} | FP16: {c['use_fp16']}")

    def train(s, td, vd):
        c = s.c; st = time.time()
        tds = DS(get_t(td), s.tok, prep_l(td), c['max_length'])
        vds = DS(get_t(vd), s.tok, prep_l(vd), c['max_length'])
        tl = DataLoader(tds, batch_size=c['batch_size'], shuffle=True, collate_fn=coll)
        vl = DataLoader(vds, batch_size=c['batch_size'], shuffle=False, collate_fn=coll)
        
        opt = AdamW(s.m.parameters(), lr=c['learning_rate'], weight_decay=c['weight_decay'])
        ts = len(tl) * c['num_epochs']
        sch = get_cosine_schedule_with_warmup(opt, int(ts * c['warmup_ratio']), ts)
        
        ba, bs, pa = 0, None, 0
        
        for ep in range(c['num_epochs']):
            t0 = time.time()
            print(f"\n{'='*60}\nEpoch {ep+1}/{c['num_epochs']}\n{'='*60}")
            s.m.train(); tls = 0
            
            for b in tqdm(tl, desc="Training"):
                ids, mask = b["input_ids"].to(s.dev), b["attention_mask"].to(s.dev)
                
                opt.zero_grad()
                
                # Mixed precision training
                if c['use_fp16']:
                    with autocast():
                        lo = s.m(ids, mask)
                        loss = sum(s.crit(lo[a], b[f"label_{a.lower()}"].to(s.dev)) for a in ASPECTS)
                    s.scaler.scale(loss).backward()
                    s.scaler.unscale_(opt)
                    torch.nn.utils.clip_grad_norm_(s.m.parameters(), 1.0)
                    s.scaler.step(opt)
                    s.scaler.update()
                else:
                    lo = s.m(ids, mask)
                    loss = sum(s.crit(lo[a], b[f"label_{a.lower()}"].to(s.dev)) for a in ASPECTS)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(s.m.parameters(), 1.0)
                    opt.step()
                
                sch.step()
                tls += loss.item()
            
            print(f"Loss: {tls/len(tl):.4f}")
            ac, dt = s._ev(vl)
            print(f"Val: {ac:.2f}% | {dt} | ⏱️ {time.time()-t0:.0f}s")
            
            if ac > ba:
                ba, bs, pa = ac, {k: v.cpu().clone() for k, v in s.m.state_dict().items()}, 0
                print("⭐ Best!")
            else:
                pa += 1
                print(f"⏳ Patience: {pa}/{c['patience']}")
                if pa >= c['patience']: print("⚠️ Early stop"); break
        
        if bs: s.m.load_state_dict(bs); s.m.to(s.dev)
        tt = time.time() - st
        print(f"\n{'='*60}\n🏆 BEST: {ba:.2f}% | ⏱️ {tt/60:.1f} min\n{'='*60}")
        return ba, tt

    def _ev(s, ld):
        s.m.eval(); cor, tot = {a: 0 for a in ASPECTS}, 0
        with torch.no_grad():
            for b in ld:
                ids, mask = b["input_ids"].to(s.dev), b["attention_mask"].to(s.dev)
                if s.c['use_fp16']:
                    with autocast():
                        lo = s.m(ids, mask)
                else:
                    lo = s.m(ids, mask)
                for a in ASPECTS:
                    cor[a] += (torch.argmax(lo[a], -1) == b[f"label_{a.lower()}"].to(s.dev)).sum().item()
                tot += ids.size(0)
        dt = {a: round(100*cor[a]/tot, 1) for a in ASPECTS}
        return sum(dt.values())/3, dt

    def predict(s, t):
        s.m.eval(); p = []
        for i in range(0, len(t), 32):
            e = s.tok(t[i:i+32], truncation=True, padding=True, max_length=s.c['max_length'], return_tensors="pt")
            with torch.no_grad():
                if s.c['use_fp16']:
                    with autocast():
                        lo = s.m(e["input_ids"].to(s.dev), e["attention_mask"].to(s.dev))
                else:
                    lo = s.m(e["input_ids"].to(s.dev), e["attention_mask"].to(s.dev))
            for j in range(len(t[i:i+32])):
                p.append({a: IDX_TO_LABEL[torch.argmax(lo[a][j]).item()] for a in ASPECTS})
        return p

print("✅ Code V9 prêt!")

In [ ]:
df_train = pd.read_csv("/content/data/ftdataset_train.tsv", sep=' *\t *', encoding='utf-8', engine='python')
df_val = pd.read_csv("/content/data/ftdataset_val.tsv", sep=' *\t *', encoding='utf-8', engine='python')
train_data, val_data = df_train.to_dict('records'), df_val.to_dict('records')
print(f"✅ Train={len(train_data)}, Val={len(val_data)}")

In [ ]:
trainer = Trainer(CONFIG)
best_acc, total_time = trainer.train(train_data, val_data)

In [ ]:
print("\n📈 Évaluation finale...")
preds = trainer.predict(get_t(val_data))
correct = {a: sum(1 for p, r in zip(preds, val_data) if p[a] == r[a]) for a in ASPECTS}
n = len(val_data)
print("\n" + "="*60)
print("📊 RÉSULTATS V9 (OPTIMISÉ)")
print("="*60)
for a in ASPECTS: print(f"  {a}: {100*correct[a]/n:.2f}%")
macro = sum(100*correct[a]/n for a in ASPECTS)/3
print(f"\n🎯 MACRO: {macro:.2f}% | ⏱️ {total_time/60:.1f} min")
print("="*60)
print(f"\n📈 Amélioration vs V8 (87.94%): {macro - 87.94:+.2f}%")

In [ ]:
torch.save({'model': trainer.m.state_dict(), 'config': CONFIG, 'acc': best_acc}, '/content/model_v9.pt')
print(f"✅ Sauvegardé (acc: {best_acc:.2f}%)")
from google.colab import files
files.download('/content/model_v9.pt')